<a href="https://colab.research.google.com/github/langroid/langroid/blob/main/examples/langroid_quick_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



<img width="700" src="https://raw.githubusercontent.com/langroid/langroid/main/docs/assets/langroid-card-lambda-ossem-rust-1200-630.png" alt="Langroid">

# Overview

This notebook provides the runnable code for the six [**Usage Examples**](https://github.com/langroid/langroid#tada-usage-examples) described in [Langroid repo](https://github.com/langroid/langroid).

**NOTE:** Notebooks (colab, jupyter, or otherwise) are *not* an ideal way to run interactive chat loops. We are showing these examples here since we recognize that Colab notebooks offer the benefit of having a ready to run environment with minimal setup. But we encourage you to try the python scripts in the [examples folder](https://github.com/langroid/langroid/tree/main/examples) of the repo on the command line for the best experience.

In the first two cells we show the steps for setting up the requirements to run the examples including the installation of `Langroid` package and setting up the `OPENAI_API_KEY`.


## Install Langroid

At the end there may be a message saying "RESTART RUNTIME", which can be safely ignored.

In [1]:
!pip install -q langroid[hf-embeddings]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.7/819.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.2/71.2 kB 8.3 MB/s et

## Set up `OPENAI_API_KEY`

This code will ask the user to provide the `OPENAI_API_KEY`. Before running this cell, please follow these steps to get the key.
Login to your OpenAI account --> go to `View API Keys` from the drop-down list on the top-right corner --> click on the botton **create new secret key** --> a new screen will pop up --> press the botton **create secret key**.

Visit [this page](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key) for more info about where to find the API Key.

In [ ]:
import os
import logging
logging.getLogger().setLevel(logging.ERROR)


from getpass import getpass

os.environ['OPENAI_API_KEY'] = getpass('Enter your OPENAI_API_KEY key: ')

**Now you can can try any of the following examples. It is recommended to go through these in sequence, although the order does NOT matter.**

---

# Direct interaction with OpenAI LLM

In this simple example, we are directly sending a message-sequence to the OpenAI `chatCompletion` API. Note that to have a multi-round converation we have to manually accumulate the dialog.

First do various imports.

In [ ]:
from langroid.language_models.base import LLMMessage, Role
from langroid.language_models.openai_gpt import (
        OpenAIGPTConfig,
        OpenAIChatModel,
        OpenAIGPT,
)
from langroid.language_models.base import LLMMessage, Role

You can use the API `OpenAIGPTConfig` to set the configurations of the OpenAI LLM model. We then define the LLM model using `OpenAIGPT`. We can also Specify the messages that will be sent to instruct the model.
`Langroid` supports various roles provided by OpenAI.


In [ ]:
cfg = OpenAIGPTConfig(chat_model=OpenAIChatModel.GPT4) # or GPT_3_5_TURBO

mdl = OpenAIGPT(cfg)

messages = [
  LLMMessage(content="You are a helpful assistant",  role=Role.SYSTEM),
  LLMMessage(content="What is the capital of Ontario?",  role=Role.USER),
]

response = mdl.chat(messages, max_tokens=200)
print("LLM response is:: ", response.message)

# accumulate messages manually

messages.append(response.to_LLMMessage())
messages.append(LLMMessage(content="what about India?", role=Role.USER))
response = mdl.chat(messages, max_tokens=200)
print("LLM response is:", response.message)

The above is a "raw" LLM interaction where you have to manage
message history. Using an Agent to wrap an LLM, and wrapping an Agent in a Task, we can set up an interactive, multi-round chat much more easily, as we show next.

## A note on the rest of the examples
In the interactive examples below, the conversation loop pauses for human input: in most cases you would hit enter (unless the example requires you to ask a question).
The interaction looks much better when run on a terminal,
and a notebook is not ideal for these. However we realize a Colab notebook does offer the benefit of having a ready to run environment.

# Define an agent, set up a task, and run it

Say you want to have a multi-round interactive chat with an LLM.

`Langroid` simplifies this process. We just need to create a `ChatAgent`, wrap it in a `Task`, and finally run the task.

In [ ]:
from langroid.agent.chat_agent import ChatAgent, ChatAgentConfig
from langroid.agent.task import Task
from langroid.language_models.openai_gpt import OpenAIChatModel, OpenAIGPTConfig

For constructing the `ChatAgent`, we need to set up some configurations:

*   No vector database will be created
*   chat model (`GPT4` or `GPT_3_5_TURBO`)

Note that `Langroid` offers specialized chatting agents such as `DocChatAgent` and `TableChatAgent`, which we will see later.

In [ ]:
config = ChatAgentConfig(
    llm = OpenAIGPTConfig(
        chat_model=OpenAIChatModel.GPT4,
    ),
    vecdb=None,
)
agent = ChatAgent(config)

A `ChatAgent` by itself offers 3 standard "responders": the LLM, the human User, and the Agent itself (e.g. to handle tool/function-calling by the LLM). To use these responders in an interactive loop, we need to wrap the Agent in a task,
and call its `run()` method.

A prompt will be displayed after running this task, so you can interact with the `ChatAgent`.

Type your questions and the agent will provide the LLM responses. When done, type `q` to exit.


In [ ]:
agent.message_history.clear()
task = Task(agent, name="Bot")
task.set_color_log(enable=False)
task.run()

# Three communicating agents

The above example involved a single `ChatAgent`, but in non-trivial applications, we will often find it easier to divide responsibilities among multiple agents, each with different skills and responsibilities.

If you attempt to solve these with a single Agent, you would have to keep track of multiple conversation states and loops, and it quickly gets out of hand. Agents offer a way to solve complex tasks in a modular fashion. Moreover, specialized agents can be designed and tested in isolation, and then combined to solve various tasks.

`Langroid` streamlines the process of setting up multiple agents and orchestrating their interaction. Here's a toy numerical example (this helps keep token costs low!). Imagine a task where we want to construct a series of numbers using the following rule to transform the current number $n$:
- if $n$ is even, the next number is $n/2$
- if $n$ is odd, the next number is $3n+1$.

We can have 3 agents collaborate to produce this sequence.
Given the current number $n$,
- `repeater_agent` simply returns $n$,
- `even_agent` specializes in handling even numbers, and returns $n/2$ if $n$ is even, else says "DO-NOT-KNOW"
- `odd_agent` specializes in handling odd numbers, and returns $3*n+1$ if $n$ is odd, else says "DO-NOT-KNOW"

In [ ]:
from langroid.utils.constants import NO_ANSWER
from langroid.agent.chat_agent import ChatAgent, ChatAgentConfig
from langroid.agent.task import Task
from langroid.language_models.openai_gpt import OpenAIChatModel, OpenAIGPTConfig

As before, we define chat model that will be used by the agents:

In [ ]:
config = ChatAgentConfig(
    llm = OpenAIGPTConfig(
        chat_model=OpenAIChatModel.GPT4,
    ),
    vecdb = None,
)

Now, we create the `repeater_agent` and define its corresponding task, which comprises the following settings:


*   **Name**: name of the agent
*   **llm_delegate**: whether to delegate control to LLM; conceptually, the "controlling entity" is the one "seeking" responses to its queries, and has a goal it is aiming to achieve. The "controlling entity" is either the LLM or the USER. (Note within a Task there is just one LLM, and all other entities are proxies of the "User" entity).
*   **single_round**: If true, the task runs until one message by the controller and a subsequent response by the non-controller. If false, runs for the specified number of turns in `run`, or until `done()` is true.
* **system_message**: provides instructions to the LLM.

In [ ]:
repeater_agent = ChatAgent(config)
repeater_task = Task(
    repeater_agent,
    name = "Repeater",
    system_message="""
    Your job is to repeat whatever number you receive.
    """,
    llm_delegate=True, # LLM takes charge of task
    single_round=False,
)

Now we define our second agent `even_agent` and its task `even_task`. Notice it takes the same `config` that we previously created.

In [ ]:
even_agent = ChatAgent(config)
even_task = Task(
    even_agent,
    name = "EvenHandler",
    system_message=f"""
    You will be given a number.
    If it is even, divide by 2 and say the result, nothing else.
    If it is odd, say {NO_ANSWER}
    """,
    single_round=True,  # task done after 1 step() with valid response
)

Finally, we create the 3rd agent `odd_agent` and its task `odd_task`.


In [ ]:
odd_agent = ChatAgent(config)
odd_task = Task(
    odd_agent,
    name = "OddHandler",
    system_message=f"""
    You will be given a number n.
    If it is odd, return (n*3+1), say nothing else.
    If it is even, say {NO_ANSWER}
    """,
    single_round=True,  # task done after 1 step() with valid response
)

We use `add_sub_task` to orchestrate the collaboration between the agents.  Specifically, `repeater_task` will act as the "main", and we add `even_task` and `odd_task` as
subtasks. For more details see these [docs](https://langroid.github.io/langroid/quick-start/multi-agent-task-delegation/#task-collaboration-via-sub-tasks).


Finally, we kickoff the task with a starting number 3, using `repeater_task.run("3")`.

Remember to keep hitting enter when it's the human's turn, and hit "q" to end the conversation.

In [ ]:
repeater_task.add_sub_task([even_task, odd_task])
repeater_task.set_color_log(enable=False)
repeater_task.run("3")

# Simple Tool/Function-calling example

Here is a simple numerical example showcasing how `Langroid` supports tools/function-calling. For more details see these [doc pages](https://langroid.github.io/langroid/quick-start/chat-agent-tool/)

Say the agent has a secret list of numbers, and we want the LLM to find the smallest number in the list. We want to give the LLM the ability to use a **probe** tool/function which takes a single number `n` as an argument. The tool handler method in the agent returns how many numbers in its list are at most `n`.

In [ ]:

from langroid.agent.tool_message import ToolMessage
from langroid.agent.chat_agent import ChatAgent, ChatAgentConfig
from langroid.language_models.openai_gpt import OpenAIChatModel, OpenAIGPTConfig
from langroid.agent.task import Task
from langroid.utils.configuration import set_global, Settings
from langroid.embedding_models.models import OpenAIEmbeddingsConfig

To use tools/function-calling in `Langroid`, we first **define** the tool as a subclass of `ToolMessage` to specify some details about the tool (e.g., name and parameters) and when it can be used/triggered:
* **request**: is the name of the tool/function, as well as the name of the Agent method that "handles" the tool.
* **purpose**: general description to give hints to LLM when this tool can be used
* **number**: is a function-argument for the `probe` tool and its type is `int`

In [ ]:
class ProbeTool(ToolMessage):
  request: str = "probe"
  purpose: str = """
        To find how many numbers in my list are less than or equal to
        the <number> you specify.
        """ # note  <number> corresponds to the name of the tool's argument/parameter
  number: int


Next, we create an agent `SpyGameAgent`, with a special method `probe` to handle the `probe` tool/function.
Notice the argument of the `probe` method is an instance of the class `ProbeTool` that we created in the previous step.

In [ ]:
class SpyGameAgent(ChatAgent):
  def __init__(self, config: ChatAgentConfig):
    super().__init__(config)
    self.numbers = [3, 4, 8, 11, 15, 25, 40, 80, 90] # agent's secret list

  def probe(self, msg: ProbeTool) -> str:
    # return how many numbers in self.numbers are less or equal to msg.number
    return str(len([n for n in self.numbers if n <= msg.number]))

Finally, we instantiate the `SpyGameAgent` as an object `spy_game_agent`, and "associate" the `probe` tool with this agent, using the `enable_message` method of the `ChatAgent`.  We then wrap the `spy_game_agent` in a `Task` object, with instructions (`system_message`) on what it should aim for.

In [ ]:

spy_game_agent = SpyGameAgent(
    ChatAgentConfig(
        llm = OpenAIGPTConfig(
            chat_model=OpenAIChatModel.GPT4,
        ),
        vecdb=None, # no vector database needed
    )
)

spy_game_agent.enable_message(ProbeTool)

task = Task(
        spy_game_agent,
        name="Spy",
        system_message="""
            I have a list of numbers between 1 and 20.
            Your job is to find the smallest of them.
            To help with this, you can give me a number and I will
            tell you how many of my numbers are equal or less than your number.
            Once you have found the smallest number,
            you can say DONE and report your answer.
        """,
    )

Now run the task.

Remember to keep hitting enter when it's the human's turn, and hit "q" to end the conversation.

In [ ]:
spy_game_agent.message_history.clear()
task.set_color_log(enable=False)
task.run()

# Chat with documents (file paths, URLs, etc)

In the previous examples, the Agents did not use any external documents. In this example, we we set up an Agent that supports "chatting" with documents. Specifically, we use the `DocChatAgent` class to ask questions about a set of URLs.
The `DocChatAgent` first ingests the contents of the websites specified by the URLs by chunking, embedding and indexing them into a vector database (`qdrant` by default). We then wrap the agent in a task and run it interactively.
The user can ask questions and the LLM of the agent returns answers using Retrieval Augment Generation, with Evidence Citation.


In [ ]:
from langroid.agent.special.doc_chat_agent import DocChatAgentConfig, DocChatAgent
from langroid.language_models.openai_gpt import OpenAIChatModel, OpenAIGPTConfig
from langroid.vector_store.qdrantdb import QdrantDBConfig
from langroid.embedding_models.models import OpenAIEmbeddingsConfig
from langroid.agent.task import Task

Now we define the configuration of the `DocChatAgent`. The configurations include the path to access the documents, chat model settings, and vector-DB settings.

In [ ]:
config = DocChatAgentConfig(
  doc_paths = [
    "https://en.wikipedia.org/wiki/Language_model",
    "https://en.wikipedia.org/wiki/N-gram_language_model",
  ],
  llm = OpenAIGPTConfig(
    chat_model=OpenAIChatModel.GPT4,
  ),
  vecdb=QdrantDBConfig(
                collection_name="docqa-chat-multi-extract",
                storage_path=".qdrant/test1/", # CHANGE THIS PATH IF YOU GET AN ERROR WHEN RE-RUNNING THE CELL
        ),
)

agent = DocChatAgent(config)

As before, we wrap the agent in a task, and run it.

Remember to keep hitting enter when it's the human's turn, and hit "q" to end the conversation.

In [ ]:
agent.message_history.clear()
task = Task(agent)
task.set_color_log(enable=False)
task.run()

# Tool/Function-calling to extract structured information from text

Let's combine multi-agent interaction, Retrieval-Augmented Generation, and tools/function-calling, for a more realistic example. Suppose you want an agent to extract the key terms of a lease, from a lease document, as a nested JSON structure.
This can be accomplished by instructing the LLM to use a specific tool.

To simplify the solution, we separate the skills/responsibilities into two different Agents:
- `LeaseExtractorAgent` has no access to the lease, and is responsible for gathering the key terms into a specific structured form
- `DocChatAgent` has access to the lease and answers specific questions it receives from the `LeaseExtractorAgent`.


In [ ]:
from rich import print
from pydantic import BaseModel, BaseSettings
from typing import List
import json

from langroid.agent.special.doc_chat_agent import DocChatAgent, DocChatAgentConfig
from langroid.agent.chat_agent import ChatAgent, ChatAgentConfig
from langroid.vector_store.qdrantdb import QdrantDBConfig
from langroid.agent.task import Task
from langroid.agent.tool_message import ToolMessage
from langroid.language_models.openai_gpt import OpenAIGPTConfig
from langroid.utils.configuration import set_global, Settings
from langroid.utils.logging import setup_colored_logging
from langroid.utils.constants import NO_ANSWER
from langroid.embedding_models.models import OpenAIEmbeddingsConfig

Next, we define the desired structure of the lease information via Pydantic models. The desired format is a nested JSON structure, which maps to a nested class structure:


In [ ]:
class LeasePeriod(BaseModel):
    start_date: str
    end_date: str

class LeaseFinancials(BaseModel):
    monthly_rent: str
    deposit: str

class Lease(BaseModel):
    """
    Various lease terms.
    Nested fields to make this more interesting/realistic
    """

    period: LeasePeriod
    financials: LeaseFinancials
    address: str

We then define the `LeaseMessage` tool as a subclass of Langroid's `ToolMessage`. The `LeaseMessage` class has a
required argument `terms` of type `Lease`. The `classmethod` named `examples` is used to generate $k$-shot examples for the LLM when instructing it to extract information in the desired structured form (see a later cell below).


In [ ]:
class LeaseMessage(ToolMessage):
    request: str = "lease_info" # maps to method of LeaseExtractorAgent
    purpose: str = """
        Collect information about a Commercial Lease.
        """
    terms: Lease

    @classmethod
    def examples(cls) -> List["LeaseMessage"]:
        return [
            cls(
                terms=Lease(
                    period=LeasePeriod(start_date="2021-01-01", end_date="2021-12-31"),
                    financials=LeaseFinancials(monthly_rent="$1000", deposit="$1000"),
                    address="123 Main St, San Francisco, CA 94105",
                ),
                result="",
            ),
        ]



Next we define the `LeaseExtractorAgent` and add a method `least_info` to handle the tool/function-call `lease_info` defined in the tool `LeaseMessage`. In this case the handling is trivial: if the method receives a valid object of class `LeaseMessage`, it declares "success".

In [ ]:
class LeaseExtractorAgent(ChatAgent):
    def __init__(self, config: ChatAgentConfig):
        super().__init__(config)

    def lease_info(self, message: LeaseMessage) -> str:
        print(
            f"""
        DONE! Successfully extracted Lease Info:
        {message.terms}
        """
        )
        return json.dumps(message.terms.dict())

In [ ]:
# Obtain the lease.txt document that we want to parsed
!wget https://github.com/langroid/langroid-examples/blob/main/examples/docqa/lease.txt

Next, set up an instance of `DocChatAgent`, point it to the lease document, equip it with a vector database, and instructions on how to answer questions based on extracts retrieved from the vector-store.


In [ ]:
doc_agent = DocChatAgent(
        DocChatAgentConfig(
            doc_paths = ["lease.txt"],
            vecdb=QdrantDBConfig(
                collection_name="docqa-chat-multi-extract",
                storage_path=".data1/data1/", # CHANGE PATH IF ERROR
              ),
            summarize_prompt= f"""
                Use the provided extracts to answer the question.
                If there's not enough information, respond with {NO_ANSWER}. Use only the
                information in these extracts, even if your answer is factually incorrect,
                and even if the answer contradicts other parts of the document. The only
                important thing is that your answer is consistent with and supported by the
                extracts. Compose your complete answer and cite all supporting sources on a
                separate separate line as "EXTRACTS:".
                Show each EXTRACT very COMPACTLY, i.e. only show a few words from
                the start and end of the extract, for example:
                EXTRACT: "The world war started in ... Germany Surrendered"
                {{extracts}}
                {{question}}
                Answer:
            """
        )
    )

Next we wrap the `doc_agent` into a Task, with instructions on its role.


In [ ]:

doc_task = Task(
    doc_agent,
    name="DocAgent",
    llm_delegate=False,
    single_round=True,
    system_message="""You are an expert on Commercial Leases.
    You will receive various questions about a Commercial
    Lease contract, and your job is to answer them concisely in at most 2 sentences.
    Please SUPPORT your answer with an actual EXTRACT from the lease,
    showing only a few words from the  START and END of the extract.
    """,
)

Finally, we instantiate the `lease_extractor_agent`, enable it to use and handle the `LeaseMessage` tool. Then we wrap the `lease_extractor_agent` into a Task, instructing it to gather information in the desired format, by asking questions one at a time. Note how the instruction contains `LeaseMessage.usage_example()`: this example is constructed from the `examples` classmethod above when the `LeaseMessage` was defined.


In [ ]:
lease_extractor_agent = LeaseExtractorAgent(
    ChatAgentConfig(
        vecdb=None,
        llm=OpenAIGPTConfig(),
    )
)

lease_extractor_agent.enable_message(
    LeaseMessage,
    use=True,
    handle=True,
    force=False,
)

lease_task = Task(
    lease_extractor_agent,
    name="LeaseExtractorAgent",
    llm_delegate=True,
    single_round=False,
    system_message=f"""
    You have to collect some information about a Commercial Lease, but you do not
    have access to the lease itself.
    You can ask me questions about the lease, ONE AT A TIME, I will answer each
    question. You only need to collect info corresponding to the fields in this
    example:
    {LeaseMessage.usage_example()}
    If some info cannot be found, fill in {NO_ANSWER}.
    When you have collected this info, present it to me using the
    'lease_info' function/tool.
    """,
)

Finally, we set up the `doc_task` as a subtask of the `lease_task` so that the `doc_agent` can respond to questions from the `lease_extractor_agent`.
 Now, the `lease_extractor_agent` will be asking questions about the lease and `doc_task` will provide the answers, citing evidence extracted from the lease. Once `lease_extractor_agent` collects all the terms of the lease as instructed, it will use the tool `LeaseMessage` to return this information.

 The next cell runs the `lease_task`. Remember to keep hitting enter when it's the human's turn, and hit "q" to end the conversation.

In [ ]:
lease_extractor_agent.message_history.clear()
lease_task.add_sub_task(doc_task)
lease_task.set_color_log(enable=False)
lease_task.run()

# Chat with tabular data (file paths, URLs, dataframes)

Here is how `Langroid's` `TableChatAgent` can be used to chat with tabular data, which can be specified as a URL, file path or Pandas dataframe.

The Agent's LLM generates Pandas code to answer the query, via function-calling (or tool/plugin), and the Agent's function-handling method executes the code and returns the answer

In [ ]:
from langroid.agent.special.table_chat_agent import TableChatAgent, TableChatAgentConfig
from langroid.agent.task import Task
from langroid.language_models.openai_gpt import OpenAIChatModel, OpenAIGPTConfig


Set up a `TableChatAgent` for a data file, URL or dataframe (Ensure the data table has a header row; the delimiter/separator is auto-detected):

In [ ]:
dataset =  "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
# or dataset = "/path/to/my/data.csv"
# or dataset = pd.read_csv("/path/to/my/data.csv")
agent = TableChatAgent(
    config=TableChatAgentConfig(
        data=dataset,
        llm=OpenAIGPTConfig(
            chat_model=OpenAIChatModel.GPT4,
        ),
    )
)

Now, let's set up a task and run it in an interactive loop with the user:
Based on `dataset`, you can ask the following question in the prompt:

```
What is the average alcohol content of wines with a quality rating above 7?
```

Remember to keep hitting enter when it's the human's turn, and hit "q" to end the conversation.

In [ ]:
agent.message_history.clear()
task = Task(agent, name="DataAssistant")
task.set_color_log(enable=False)
task.run()